In [ ]:
!pip install scikit-learn==0.22.1

In [ ]:
!pip show imbalanced-learn

In [ ]:
pip install kaggler

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import random
import glob

import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import log_loss

from kaggler.model import AutoLGB
from kaggler.preprocessing import LabelEncoder

from pprint import pprint

from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
import six
import sys
sys.modules['sklearn.externals.six'] = six
from sklearn.externals import six
from imblearn.over_sampling import SMOTE

import matplotlib.pyplot as plt

In [ ]:
CV Log Loss: 0.750443
2    0.8969
1    0.0906
0    0.0125

In [ ]:
def days_to_age(x) :
    return int(((x*-1)/365)+1)
    
def months_to_year(x) :
    return int(((x*-1)/12)+1)

def days_to_month(x) :
    return int(((x*-1)/30)+1)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
PATH = './drive/MyDrive/Credit/data/'
train = pd.read_csv(PATH+'train.csv')
test = pd.read_csv(PATH+'test.csv')
submit = pd.read_csv(PATH+'sample_submission.csv')

train.drop(['index', 'FLAG_MOBIL'], axis=1, inplace=True)

* Credit외 완전 중복 상태의 데이터가 3155개 존재
    * 다행히 완전 중복의 경우 credit이 변하지 않았음

In [ ]:
# 완전 중복 확인
train['iden'] = train.apply(lambda x : "".join([str(x[i]) for i in list(set(train.columns)-set('credit'))]), axis=1)

# for i in range(len(train['iden'].value_counts()[train['iden'].value_counts()>1].index)):
#     if train[train['iden'] == train['iden'].value_counts()[train['iden'].value_counts()>1].index[i]]['credit'].nunique() != 1:
#         print(i)

dupli_df = train.loc[train['iden'].isin(train['iden'].value_counts()[train['iden'].value_counts()>1].index)]
drop_list = list(set(dupli_df.index) - set(dupli_df.drop_duplicates().index))

# 중복 데이터 중 첫 번째 데이터 제외 삭제
train = train[~train.index.isin(drop_list)]

* Credit외 완전 중복 상태의 데이터가 3155개 존재
    * 다행히 완전 중복의 경우 credit이 변하지 않았음

* 내 생각에는 income_type이 유의미하지 않은 변수인 듯

In [ ]:
train['credit'].value_counts() / train['credit'].count(),\
train.loc[train['income_type']=='State servant']['credit'].value_counts() / train.loc[train['income_type']=='State servant']['credit'].count(),
train['income_type'].unique()

In [ ]:
train['identity'] = [str(i)+str(j)+str(k)+str(l)+str(m) for i,j,k,l,m in zip(train['gender'], train['DAYS_BIRTH'], train['DAYS_EMPLOYED'], train['edu_type'], train['income_total'])]


In [ ]:
dupli_col = train['identity'].value_counts().index

In [ ]:
surv_list = []
for i in range(len(dupli_col)) :
    surv_list.append(train.loc[train['identity']==dupli_col[i]].sort_values(['begin_month']).drop_duplicates('credit',keep='first').index[0])

In [ ]:
train = train[train.index.isin(surv_list)]

In [ ]:
test.drop(['index', 'FLAG_MOBIL'], axis=1, inplace=True)

train.loc[train['DAYS_EMPLOYED']>0, 'DAYS_EMPLOYED'] = 0
test.loc[test['DAYS_EMPLOYED']>0, 'DAYS_EMPLOYED'] = 0

# occyp_type을 쓸거냐 안쓸거냐의 차이 / 없을 때가 성능이 더 뛰어났긴 함
train.loc[train['DAYS_EMPLOYED'] == 0, 'occyp_type'] = 'Unemployed'
test.loc[test['DAYS_EMPLOYED'] == 0, 'occyp_type'] = 'Unemployed'
train.dropna(inplace=True)

train = train[train['child_num'] < train['family_size']]






train.reset_index(drop=True, inplace=True)

df = pd.concat([train.drop('credit', axis=1), test])

In [ ]:
PATH = './drive/MyDrive/Credit/data/'
train = pd.read_csv(PATH+'train.csv')
test = pd.read_csv(PATH+'test.csv')
submit = pd.read_csv(PATH+'sample_submission.csv')



train.drop(['index', 'FLAG_MOBIL'], axis=1, inplace=True)
test.drop(['index', 'FLAG_MOBIL'], axis=1, inplace=True)

train.loc[train['DAYS_EMPLOYED']>0, 'DAYS_EMPLOYED'] = 0
test.loc[test['DAYS_EMPLOYED']>0, 'DAYS_EMPLOYED'] = 0

# occyp_type을 쓸거냐 안쓸거냐의 차이 / 없을 때가 성능이 더 뛰어났긴 함
train.loc[train['DAYS_EMPLOYED'] == 0, 'occyp_type'] = 'Unemployed'
test.loc[test['DAYS_EMPLOYED'] == 0, 'occyp_type'] = 'Unemployed'
train.dropna(inplace=True)

train = train[train['child_num'] < train['family_size']]






train.reset_index(drop=True, inplace=True)

df = pd.concat([train.drop('credit', axis=1), test])
# df.dropna(inplace=True)
# df.fillna('Unknown', inplace=True)

In [ ]:
CV Log Loss: 0.736840
2    0.8693
1    0.1058
0    0.0249

In [ ]:
df['age_gen'] = df['DAYS_BIRTH'].apply(lambda x : '60' if days_to_age(x)//10*10 >= 60 else ( '10' if days_to_age(x)//10*10 < 20 else str(days_to_age(x)//10*10)))+ '_' + df['gender']
df['phone_mail'] = df.apply(lambda x : str(x['phone'])+'_'+str(x['email'])+'_'+str(x['work_phone']), axis=1)
df['real_car'] = df.apply(lambda x : str(x['reality'])+'_'+str(x['car']), axis=1)

df.drop(['gender', 'email', 'phone', 'car','reality', 'work_phone'], axis=1, inplace=True)

In [ ]:
# test['identity'] = [str(i)+str(j)+str(k)+str(l)+str(m) for i,j,k,l,m in zip(test['gender'], test['DAYS_BIRTH'], test['DAYS_EMPLOYED'], test['edu_type'], test['income_total'])]

In [ ]:
# df.loc[df['DAYS_EMPLOYED']>=0, 'DAYS_EMPLOYED'] = 0
# df.loc[df['DAYS_EMPLOYED']==0, 'occyp_type'] = 'Unemployed'

df['before_EMPLOYED']=df['DAYS_BIRTH']-df['DAYS_EMPLOYED']
df['before_EMPLOYED_month']=np.floor((-df['before_EMPLOYED'])/30)-((np.floor((-df['before_EMPLOYED'])/30)/12).astype(int)*12)
df['before_EMPLOYED_week']=np.floor((-df['before_EMPLOYED'])/7)-((np.floor((-df['before_EMPLOYED'])/7)/4).astype(int)*4)

df['DAYS_BIRTH_month']=np.floor((-df['DAYS_BIRTH'])/30)-((np.floor((-df['DAYS_BIRTH'])/30)/12).astype(int)*12)
df['DAYS_BIRTH_week']=np.floor((-df['DAYS_BIRTH'])/7)-((np.floor((-df['DAYS_BIRTH'])/7)/4).astype(int)*4)

df['DAYS_EMPLOYED_month']=np.floor((-df['DAYS_EMPLOYED'])/30)-((np.floor((-df['DAYS_EMPLOYED'])/30)/12).astype(int)*12)
df['DAYS_EMPLOYED_week']=np.floor((-df['DAYS_EMPLOYED'])/7)-((np.floor((-df['DAYS_EMPLOYED'])/7)/4).astype(int)*4)

df['DAYS_BIRTH'] = -df['DAYS_BIRTH']
df['DAYS_EMPLOYED'] = -df['DAYS_EMPLOYED']
df['begin_month'] = -df['begin_month']
df['before_EMPLOYED'] = - df['before_EMPLOYED']

df['begin_credit'] = df['DAYS_EMPLOYED'] - 30*df['begin_month']
# df['BEFORE_EMPLOYED'] = df['BEFORE_EMPLOYED'].apply(lambda x : days_to_age(x))
# df['DAYS_BIRTH'] = df['DAYS_BIRTH'].apply(lambda x : days_to_age(x))
# df['DAYS_EMPLOYED'] = df['DAYS_EMPLOYED'].apply(lambda x : days_to_month(x))

# df[['work_phone', 'phone', 'email']] = df[['work_phone', 'phone', 'email']].astype('object')

edu_dict = {'Academic degree' : 4, # 학사 이상
            'Higher education' : 3, # 고등학교 졸업
            'Incomplete higher' : 2, # 고등학교 중퇴
           'Secondary / secondary special' : 1, # 중학교
            'Lower secondary' : 0} # 중학교 미만

df['edu_type'] = df['edu_type'].apply(lambda x: edu_dict[x])

# df['temp'] = df['family_size'] - df['child_num']

이번에는, Family Type를 확인해보자.
Married의 비중이 압도적으로 많다.
카드사 데이터 특징 중, 실질적인 카드 소유권이 남성보다는 여성이 많고, 미혼보다는 기혼이 더 많다는 것을 확인할 수 있다.

이번에는, Education Type를 확인해보자.
Credit 2번 그룹을 중심으로 Secondary / Secondary Special 그룹과 Higher education 그룹에서 많은 빈도가 나타나고 있다.

In [ ]:
ob_col = [x for x in df.columns if df[x].dtype == 'object']
# label_col = ob_col[:3] + ['age_gen']
# ordinal_col = ['income_type', 'family_type', 'house_type', 'occyp_type']

In [ ]:
lbe = LabelEncoder(min_obs=10)
df[ob_col] = lbe.fit_transform(df[ob_col])

In [ ]:
ordinal_col = ['age_gen', 'phone_mail', 'real_car']

In [ ]:
# Binarize
import numpy as np
from scipy import sparse

def binarize_target(target_col):
    n = df[target_col].nunique()
    print('number of cateories : ', n)
    binarize_labels = sorted(df[target_col].unique())
    print('binarize target labels : ', binarize_labels)
    k = 0
    while n > 2**k:
        k += 1
    print('necessary number for binarize : ', k)

    binarized_result = np.zeros((df.shape[0],k))

    binarized_values = df[target_col].apply(lambda x : list(map(int,bin(x).replace('0b','').zfill(k)))).values
    for index, value in enumerate(binarized_values):
        binarized_result[index] = np.array(value)

    binarized_result = pd.DataFrame(binarized_result, columns = [target_col+'_'+str(x) for x in range(k)],
                                    index=df.index)
    return binarized_result

for ordinal in ordinal_col : 
    binar_df = binarize_target(ordinal)
    df = pd.concat([df, binar_df], axis=1).drop(columns=[ordinal])

In [ ]:
import scipy.stats as ss

normalize_list = ['income_total', 'DAYS_BIRTH', 'DAYS_EMPLOYED'] + ['begin_month', 'before_EMPLOYED', 'before_EMPLOYED_month', 'before_EMPLOYED_week', 'DAYS_BIRTH_month', 'DAYS_BIRTH_week', 'DAYS_EMPLOYED_month', 'DAYS_EMPLOYED_week', 'begin_credit']
normal_df = pd.DataFrame(ss.zscore(df[normalize_list]), columns=normalize_list, index=df.index)
df = pd.concat([df.drop(columns=normalize_list, axis=1), normal_df], axis=1)

In [ ]:
df

In [ ]:
y = train['credit']
n_trn = train.shape[0]
X = df.iloc[:n_trn]
X_tst = df.iloc[n_trn:]

In [ ]:
# scaler = MinMaxScaler(feature_range=(0, 1))
# # scaler.fit_transform(X)
# X = scaler.fit_transform(X)

In [ ]:
sm = SMOTE(ratio = 'auto', kind='regular')

X_resampled, y_resampled = sm.fit_sample(X, list(y))

print('After OverSampling, the shape of train_X: {}'.format(X_resampled.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(X_resampled.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_resampled==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_resampled==0)))
print("After OverSampling, counts of label '2': {}".format(sum(y_resampled==2)))

In [ ]:
X = pd.DataFrame(X_resampled, columns=X.columns)
X['credit'] = y_resampled

In [ ]:
# X = pd.DataFrame(X, columns=df.columns)

In [ ]:
y = X['credit']
X = X.drop(['credit'], axis=1)
# n_trn = train.shape[0]
# # X = df.iloc[:n_trn]
# X_tst = df.iloc[n_trn:]

n_class = 3
n_fold = 5


cv = StratifiedKFold(n_splits=n_fold)

p = np.zeros((X.shape[0], n_class), dtype=float)
p_tst = np.zeros((X_tst.shape[0], n_class), dtype=float)
# params = {'num_class': n_class}
params = {'bagging_fraction': 0.7000000000000001, 'bagging_freq': 1, 'boosting': 'gbdt', 'feature_fraction': 0.5,\
          'feature_pre_filter': False, 'lambda_l1': 0, 'lambda_l2': 0.1, 'learning_rate': 0.020391961848274073,\
          'max_depth': -1, 'metric': 'multi_logloss', 'min_child_samples': 10, 'num_class': 3, 'num_leaves': 255,\
          'num_threads': -1, 'objective': 'multiclass', 'seed': 42, 'verbosity': -1}
# features = ['gender', 'car', 'reality', 'child_num', 'income_total', 'edu_type', 'DAYS_BIRTH', 'DAYS_EMPLOYED',\
#             'work_phone', 'phone', 'email', 'family_size', 'begin_month', 'before_EMPLOYED', 'before_EMPLOYED_month',\
#             'before_EMPLOYED_week', 'DAYS_BIRTH_month', 'DAYS_BIRTH_week', 'DAYS_EMPLOYED_month', 'DAYS_EMPLOYED_week',\
#             'income_type_0', 'income_type_1', 'income_type_2', 'family_type_0', 'family_type_1', 'family_type_2',\
#             'house_type_0', 'house_type_1', 'house_type_2', 'occyp_type_0', 'occyp_type_1', 'occyp_type_2', 'occyp_type_3', 'occyp_type_4']
features = df.columns
n_best = 206

# params = {'bagging_freq': 1, 'verbosity': -1, 'seed': 42, 'num_threads': -1, 'feature_pre_filter': False, 'num_class': 3, 'objective': 'multiclass', 'metric': 'multi_logloss', 'boosting': 'gbdt', 'bagging_fraction': 0.9, 'feature_fraction': 0.7000000000000001, 'lambda_l1': 0, 'lambda_l2': 0.1, 'learning_rate': 0.23323579910038686, 'max_depth': 4, 'min_child_samples': 25, 'num_leaves': 15}
# features = ['gender', 'car', 'reality', 'child_num', 'income_total', 'income_type', 'edu_type', 'family_type', 'house_type', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'work_phone', 'phone', 'email', 'occyp_type', 'family_size', 'begin_month', 'before_EMPLOYED', 'before_EMPLOYED_month', 'before_EMPLOYED_week', 'DAYS_BIRTH_month', 'DAYS_BIRTH_week', 'DAYS_EMPLOYED_month', 'DAYS_EMPLOYED_week']
# n_best = 871

for i_cv, (i_trn, i_val) in enumerate(cv.split(X, y)):
    # if i_cv == 0:
    #     clf = AutoLGB(objective='multiclass', metric='multi_logloss', params=params, 
    #                   feature_selection=False, n_est=10000)
    #     clf.tune(X.iloc[i_trn], y[i_trn])
    #     n_best = clf.n_best
    #     features = clf.features
    #     params = clf.params
    #     print(f'best iteration: {n_best}')
    #     print(f'selected features ({len(features)}): {features}')        
    #     print(params)
    #     clf.fit(X.iloc[i_trn], y[i_trn])
    # else:
    #     train_data = lgb.Dataset(X[features].iloc[i_trn], label=y[i_trn])
    #     clf = lgb.train(params, train_data, n_best, verbose_eval=100)
    
    # p[i_val] = clf.predict(X[features].iloc[i_val])
    # p_tst += clf.predict(X_tst[features]) / n_fold

    train_data = lgb.Dataset(X[features].iloc[i_trn], label=y[i_trn])
    clf = lgb.train(params, train_data, n_best, verbose_eval=100)
    
    p[i_val] = clf.predict(X[features].iloc[i_val])
    p_tst += clf.predict(X_tst[features]) / n_fold

In [ ]:
df.columns

In [ ]:
clf.feature_importance()

In [ ]:
from sklearn.metrics import log_loss
print(f'CV Log Loss: {log_loss(y, p):f}')

In [ ]:
submit = pd.read_csv(PATH+'sample_submission.csv')

submit[submit.columns[1:]] = p_tst
# submit.to_csv('20210514_03_autolgb.csv', index=False)

In [ ]:
def get_ans(df) :
    df.drop(['index'], axis=1, inplace=True)
    temp = pd.Series(df.apply(lambda x : 0 if x['0'] == max(x) else ( 1 if x['1'] == max(x) else 2),  axis=1))
    display(temp.value_counts()/len(temp))

In [ ]:
get_ans(submit)

In [ ]:
0.6667664914730588

In [ ]:
rank1 = pd.read_csv('/20210513_03_autolgb.csv').drop(['index'], axis=1)
rank2 = pd.read_csv('/20210513_submit_binary_autolgb.csv').drop(['index'], axis=1)
rank3 = pd.read_csv('/20210513_submit_autolgb.csv').drop(['index'], axis=1)
rank4 = pd.read_csv('/20210514_01_smote_autolgb.csv').drop(['index'], axis=1)
rank5 = pd.read_csv('/20210514_03_autolgb.csv').drop(['index'], axis=1)
rank6 = pd.read_csv('/20210514_02_autolgb.csv').drop(['index'], axis=1)

In [ ]:
get_ans(submit.drop(['index'], axis=1))

2.0    0.641343  
1.0    0.236875  
0.0    0.121783  


In [ ]:
get_ans(rank1), get_ans(rank2), get_ans(rank3), get_ans(rank4), get_ans(rank5), get_ans(rank6)